# EXEMPLO 2 - Rede Neural. Estudo de Caso: Mnist.

Neste exemplo, vamos utilizar uma Rede Neural para resolver um problema clássico de classificação: o reconhecimento de dígitos escritos à mão, utilizando o dataset MNIST.

O MNIST é um conjunto de dados muito conhecido na área de Machine Learning.
Ele contém milhares de imagens em preto e branco de dígitos de 0 a 9, escritas à mão,
e o objetivo é treinar um modelo que consiga identificar corretamente qual número aparece em cada imagem.

Vamos usar uma Rede Neural simples (usando PyTorch) para aprender os padrões dessas imagens
e, depois do treinamento, testar sua capacidade de reconhecer novos dígitos.

Esse exemplo é muito útil para entender como redes neurais funcionam em problemas visuais
e serve como porta de entrada para tarefas mais complexas de Visão Computacional.

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets
from torch import nn, optim